<h1><center>Kehkashan 9825 Quiz#2</center></h1>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("pySpWordCountCSV").getOrCreate()

e:\Course\Bigdata\Quiz\q1\env\lib\site-packages\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
country = spark.read.csv("./data/Country.csv",inferSchema=True, header = True)
sales = spark.read.csv("./data/Sales.csv",inferSchema=True, header = True)


In [5]:
sales.show()

+---------+----------+---------------+---------+---------+--------+------+
|  OrderID| OrderDate|       ItemType|UnitsSold|UnitPrice|UnitCost|CtryCd|
+---------+----------+---------------+---------+---------+--------+------+
|669165933| 5/28/2010|      Baby Food|     9925|   255.28|  159.42|    TV|
|963881480| 8/22/2012|         Cereal|     2804|    205.7|  117.11|    GD|
|341417157|  5/2/2014|Office Supplies|     1779|   651.21|  524.96|    RU|
|514321792| 6/20/2014|         Fruits|     8102|     9.33|    6.92|    ST|
|115456712|  2/1/2013|Office Supplies|     5062|   651.21|  524.96|    RW|
|547995746|  2/4/2015|      Baby Food|     2974|   255.28|  159.42|    SB|
|135425221| 4/23/2011|      Household|     4187|   668.27|  502.54|    AO|
|871543967| 7/17/2012|     Vegetables|     8082|   154.06|   90.93|    BF|
|770463311| 7/14/2015|  Personal Care|     6070|    81.73|   56.67|    CG|
|616607081| 4/18/2014|         Cereal|     6593|    205.7|  117.11|    SN|
|814711606| 6/24/2011|   

In [6]:
country.show()

+------+--------------------+--------------------+
|CtryCd|             Country|              Region|
+------+--------------------+--------------------+
|    AL|             Albania|              Europe|
|    AO|              Angola|  Sub-Saharan Africa|
|    AT|             Austria|              Europe|
|    AU|           Australia|Australia and Oce...|
|    AZ|          Azerbaijan|Middle East and N...|
|    BD|          Bangladesh|                Asia|
|    BF|        Burkina Faso|  Sub-Saharan Africa|
|    BG|            Bulgaria|              Europe|
|    BN|              Brunei|                Asia|
|    BZ|              Belize|Central America a...|
|    CD|Congo, the Democr...|  Sub-Saharan Africa|
|    CG|Republic of the C...|  Sub-Saharan Africa|
|    CH|         Switzerland|              Europe|
|    CI|       Cote d'Ivoire|  Sub-Saharan Africa|
|    CM|            Cameroon|  Sub-Saharan Africa|
|    CR|          Costa Rica|Central America a...|
|    CV|          Cape Verde|  

# Join Both Tables

In [7]:
salesCountry = sales.join(country, sales.CtryCd == country.CtryCd,'left')
salesCountry.show()

+---------+----------+---------------+---------+---------+--------+------+------+--------------------+--------------------+
|  OrderID| OrderDate|       ItemType|UnitsSold|UnitPrice|UnitCost|CtryCd|CtryCd|             Country|              Region|
+---------+----------+---------------+---------+---------+--------+------+------+--------------------+--------------------+
|669165933| 5/28/2010|      Baby Food|     9925|   255.28|  159.42|    TV|    TV|              Tuvalu|Australia and Oce...|
|963881480| 8/22/2012|         Cereal|     2804|    205.7|  117.11|    GD|    GD|             Grenada|Central America a...|
|341417157|  5/2/2014|Office Supplies|     1779|   651.21|  524.96|    RU|    RU|              Russia|              Europe|
|514321792| 6/20/2014|         Fruits|     8102|     9.33|    6.92|    ST|    ST|Sao Tome and Prin...|  Sub-Saharan Africa|
|115456712|  2/1/2013|Office Supplies|     5062|   651.21|  524.96|    RW|    RW|              Rwanda|  Sub-Saharan Africa|
|5479957

# Calculate Sales Amount by multiplying Price by Units Sold

In [8]:
salesAmount = salesCountry.withColumn("salesAmount", sales["UnitPrice"] * sales["UnitsSold"])

In [9]:
salesAmount.show()

+---------+----------+---------------+---------+---------+--------+------+------+--------------------+--------------------+------------------+
|  OrderID| OrderDate|       ItemType|UnitsSold|UnitPrice|UnitCost|CtryCd|CtryCd|             Country|              Region|       salesAmount|
+---------+----------+---------------+---------+---------+--------+------+------+--------------------+--------------------+------------------+
|669165933| 5/28/2010|      Baby Food|     9925|   255.28|  159.42|    TV|    TV|              Tuvalu|Australia and Oce...|         2533654.0|
|963881480| 8/22/2012|         Cereal|     2804|    205.7|  117.11|    GD|    GD|             Grenada|Central America a...| 576782.7999999999|
|341417157|  5/2/2014|Office Supplies|     1779|   651.21|  524.96|    RU|    RU|              Russia|              Europe|        1158502.59|
|514321792| 6/20/2014|         Fruits|     8102|     9.33|    6.92|    ST|    ST|Sao Tome and Prin...|  Sub-Saharan Africa|          75591.66|

# Calculate the Profit/Loss of each Item Type

In [10]:
salesWithProfit = salesAmount.withColumn("totalCost", salesAmount["UnitsSold"] * salesAmount["UnitCost"])
salesWithProfit = salesWithProfit.withColumn("profit", salesWithProfit["salesAmount"] - salesWithProfit["totalCost"])

In [11]:
salesWithProfit.show()

+---------+----------+---------------+---------+---------+--------+------+------+--------------------+--------------------+------------------+------------------+------------------+
|  OrderID| OrderDate|       ItemType|UnitsSold|UnitPrice|UnitCost|CtryCd|CtryCd|             Country|              Region|       salesAmount|         totalCost|            profit|
+---------+----------+---------------+---------+---------+--------+------+------+--------------------+--------------------+------------------+------------------+------------------+
|669165933| 5/28/2010|      Baby Food|     9925|   255.28|  159.42|    TV|    TV|              Tuvalu|Australia and Oce...|         2533654.0|1582243.4999999998| 951410.5000000002|
|963881480| 8/22/2012|         Cereal|     2804|    205.7|  117.11|    GD|    GD|             Grenada|Central America a...| 576782.7999999999|         328376.44|248406.35999999993|
|341417157|  5/2/2014|Office Supplies|     1779|   651.21|  524.96|    RU|    RU|              

# Calculate average sales per region

In [12]:
averageSalesPerCountry = salesWithProfit.groupBy("Region").avg("salesAmount")

In [13]:
averageSalesPerCountry.show()

+--------------------+------------------+
|              Region|  avg(salesAmount)|
+--------------------+------------------+
|Middle East and N...| 1312700.798181818|
|Australia and Oce...|1281296.8300000003|
|              Europe|1516769.6413636364|
|  Sub-Saharan Africa|1102000.8730555559|
|Central America a...|1463897.2149999999|
|       North America|1881118.8500000003|
|                Asia|1940644.6381818184|
+--------------------+------------------+



# Calculate the number of Item Types per country

In [14]:
itemsPerCountry = salesWithProfit.groupBy("Country").count()

In [15]:
itemsPerCountry.show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|              Russia|    1|
|             Senegal|    1|
|Congo, the Democr...|    1|
|            Kiribati|    1|
|            Djibouti|    3|
|            Malaysia|    1|
|                Fiji|    1|
|             Comoros|    1|
|              Rwanda|    2|
|              France|    1|
|           Sri Lanka|    1|
|            Slovakia|    1|
|              Angola|    1|
|            Moldova |    1|
|          San Marino|    1|
|          East Timor|    1|
|             Lesotho|    1|
|             Albania|    1|
|          Madagascar|    1|
|             Myanmar|    2|
+--------------------+-----+
only showing top 20 rows



# Get common Item Types in tow regions (select any two regions)

In [44]:
subsharan=salesWithProfit.select(['Region','ItemType']).where(salesWithProfit['Region']=='Sub-Saharan Africa')
europe=salesWithProfit.select(['Region','ItemType']).where(salesWithProfit['Region']=='Europe')
subDf1 = subsharan.select("ItemType")
subDf2 = europe.select("ItemType")

In [45]:
common = subDf1.intersect(subDf2)
common.show()

+---------------+
|       ItemType|
+---------------+
|      Baby Food|
|      Household|
|     Vegetables|
|      Beverages|
|Office Supplies|
|      Cosmetics|
|  Personal Care|
|        Clothes|
+---------------+



In [46]:
common = subDf2.intersect(subDf1)
common.show()

+---------------+
|       ItemType|
+---------------+
|      Baby Food|
|      Household|
|     Vegetables|
|      Beverages|
|Office Supplies|
|      Cosmetics|
|  Personal Care|
|        Clothes|
+---------------+

